In [1]:
from agents.DistillPPO import DistillPPOAgent
from common.hyperparameters import HYPERPARAMS
from common.env.env import make_env 
import torch
import wandb
import numpy as np

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rullofederico16 (fede-). Use `wandb login --relogin` to force relogin


True

In [2]:
game = 'leaper'
model = 'distill'
params = HYPERPARAMS['distill']
log = wandb.init(project=f"procgen-ppo-test-{game}-{params.run_name}", config=params)
env = make_env(game, params=params)

In [3]:
agent = DistillPPOAgent(env.observation_space.shape, env.action_space.n,params)
agent.net.load_state_dict(torch.load(f"checkpoints/best_{model}_{game}.dat"))
agent.net.train(False)
env = make_env(game,params=params, test=True, viz=False, teacher_model=agent.distill_teacher, student_model=agent.distill_student, sum_rewards=False)
tot_rew = []
for _ in range(20000):
    rewards = 0.0
    steps = 0
    obs = env.reset()
    while True:
        action, _, _,_ = agent.select_action(obs)
        action = action[0]
        obs, reward, done, _ = env.step(action)
        rewards += reward[0]
        steps += 1
        if done:
            break
    tot_rew.append(rewards)
    wandb.log({'test_reward': rewards, 'test_steps': steps})
print(f"Mean reward: {np.mean(tot_rew)}")

/tmp/ipykernel_99610/652076410.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent.net.load_state_dict(torch.load(f"checkpoints/best_{model}_{game}.dat"))


RuntimeError: Error(s) in loading state_dict for DistillPPONet:
	Unexpected key(s) in state_dict: "critic_int.4.weight", "critic_int.4.bias". 
	size mismatch for critic_int.2.weight: copying a param with shape torch.Size([512, 256]) from checkpoint, the shape in current model is torch.Size([1, 256]).
	size mismatch for critic_int.2.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([1]).